In [14]:
import os
from glob import glob
name_pairs={

"taskB_organizer":"clinicalnlp_taskB_test1",
"taskC_organizer":"clinicalnlp_taskC_test2",
"clef_organizer":"clef_taskC_test3",
"chatgpt_run2":"ChatGPT_",
"chatgpt_run2":"ChatGPT_",
"davinci2_run2":"Text-Davinci-002_",
"davinci3_run2":"Text-Davinci-003_",
"gpt4_run2":"GPT-4_",
"clinicalnlp_taskB_test1":"test1",
"clinicalnlp_taskC_test2":"test2",
"clef_taskC_test3":"test3",
"_1024":"",
"_gl1024":"",

}

files=glob("../baselines/predictions/*.csv")
files+=glob("../results/*.json")
for file in files:
    new_name=file+""
    for key in name_pairs:
        new_name=new_name.replace(key,name_pairs[key])
    os.rename(file,new_name)

In [38]:
import os
for file in glob("/home/velvinfu/code/aci-demo-benchmark-private-main/results/*.csv.json"):
    result=json.loads(open(file).read())["ALL"]["rouge1"]
    if result>0.99:
        print(file)

In [32]:
from glob import glob
import json
import os
files=glob("../results/*.csv.json")
#testsets=["clinicalnlp_taskB_test1","clinicalnlp_taskC_test2","clef_taskC_test3"] #"train","valid",
testsets=["test1","test2","test3"]
# test 1
for dataset in testsets:
    model_map={
        "longest spearker turn":'../results/longest_speaker_turn_{}.csv.json'.format(dataset),
        "longest doctor turn":'../results/longest_doctor_turn_{}.csv.json'.format(dataset),
        "12 speaker turns":'../results/12_speaker_turns_{}.csv.json'.format(dataset),
        "12 doctor turns": '../results/12_doctor_turns_{}.csv.json'.format(dataset),
        "transcript":"../results/transcript_{}.csv.json".format(dataset),
        "train_UMLS":"../results/UMLS_similarity_{}.csv.json".format(dataset),
        "train_sent":'../results/spacy_similarity_{}.csv.json'.format(dataset),
    }

    #read_data
    for model in ["BART_large","bart-large-xsum-samsum","BioBART","LED","LED_pubmed",
                  "Text-Davinci-002","Text-Davinci-003","ChatGPT","GPT-4"]:
        outfile='../results/{}_{}_full_ori.csv.json'.format(model,dataset)
        #open_ai models
        if not os.path.isfile(outfile):
            outfile='../results/{}{}_.csv.json'.format(dataset,model) 
        if outfile not in files:
            outfile=[file for file in files if "{}_{}_full_ori".format(model,dataset) in file]
            outfile.sort()
            outfile=outfile[-1] if outfile else ""
        key=model.replace("_1024","").replace("bart-large","BART").replace("_large","").replace("-xsum-samsum","+FT_$_{\mathrm{SAMSum}}$").replace("_pubmed","+FT$_{\mathrm{PubMed}}$")
        #baseline
        model_map[key]=outfile

        if model in ["BART_large","bart-large-xsum-samsum","BioBART","LED","LED_pubmed"]:
            #Division
            outfile=[file for file in files if model in file and "{}_{}_ori_SOAP_combined".format(model,dataset) in file]
            model_map[key+" (Division)"]=outfile[-1] if outfile else ""
        
    #map
    with open("../tables/{}.csv".format(dataset),"w") as f:
        for result_type in ["ALL", "division-subjective","division-objective_exam",
                            "division-objective_results","division-assessment_and_plan"]:
            f.write("\n\n"+result_type+"\n")
            headers=['Model','ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'BERTScore', 'BLEURT','Fact',"average"]
            f.write(",".join(headers)+"\n")
            for key in model_map:
                if key in ["BART","LED"]:
                    f.write("{}-based\n".format(key))
                elif key=="longest spearker turn":
                    f.write("Transcript-based\n")
                elif key=="train_UMLS":
                    f.write("Retrieval-based\n")
                elif key=="Text-Davinci-002":
                    f.write("OpenAI (wo FT)\n")
                row=[key]
                if os.path.isfile(model_map[key]):
                    result=json.loads(open(model_map[key]).read())[result_type]
                    for metric in ['rouge1', 'rouge2', 'rougeLsum', 'bertscore-f1', 'bleurt','umls']:
                        row.append(float(result[metric]))
                    row.append(((sum(row[1:4])/3+sum(row[4:7])))/4)
                    row=[row[0]]+["{:.2f}".format(round(r*100,2)) for r in row[1:]]
                else:
                    row=row+["NA"]*7
                f.write(",".join(row)+"\n")
            # f.write("OpenAI (wo FT)\n")
            # f.write("\n".join(["Text-Davinci-00{} [p{}]".format(t1,t2) for t1 in [2,3] for t2 in [1,2]]))
            # f.write("\nChatGPT (gpt-3.5-turbo) [p1]\nChatGPT (gpt-3.5-turbo) [p2]\n")
        # print('\nTexttable Table:')
        # print(table.draw())

In [22]:
open(model_map[key]).read()

'../results/bart-large-xsum-samsum_test1_ori_SOAP_combined.json'

In [ ]:
'../results/{}_{}_full_ori_prediction_{}_full.csv.json'.format(model,dataset,dataset)

'/home/velvinfu/code/clef2023-internal/results/BART_large_valid_full_ori_prediction_valid_full.csv.json'

# ablation study

In [15]:
##ablation study

from glob import glob
import json
import os
files=glob("../results/*.json")

all_models=["aci_asr","aci_asrcorr"] #["virtscribe_asr", "virtscribe_humantrans"]
train_set="on_aci_asrcorr" #["ori","on_virtscribe_asr"]

all_models=["virtscribe_asr", "virtscribe_humantrans"]
train_set="on_virtscribe_asr"

for models in [all_models]:
    # test 1
    with open("../tables/{}.csv".format("_vs_".join(models)),"w") as f:
        f.write("evaluation of the model on the full train set (division-based) on all the ablation datasets\n")
        headers=['Model','ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'BERTScore', 'BLEURT','Fact',"Average"]
        f.write(",".join(headers)+"\n")
        for dataset in ["test1","test2","test3"]:
            f.write("{},".format(dataset))
            model_map={}
            #read_data
            for model in models:
                #dataset,dataset,
                outfile='../results/bart-large-xsum-samsum_ablation_{}_{}_train_ori_division_combined.csv.json'.format(dataset,model)
                
                if not os.path.isfile(outfile):
                    outfile='../results/bart-large-xsum-samsum_ablation_{}_{}_train_ori_division_combined.csv.json'.format(dataset,model)
                model_map[model+"-ori"]=outfile
                outfile='../results/bart-large-xsum-samsum_ablation_SOAP_note_train_{}_{}_{}.csv.json'.format(train_set,dataset,model)
                model_map[model+"-"+train_set]=outfile

            for key in model_map:
                    row=[key]
                    if os.path.isfile(model_map[key]):
                        result=json.loads(open(model_map[key]).read())["ALL"]
                        for metric in ['rouge1', 'rouge2', 'rougeLsum', 'bertscore-f1', 'bleurt','umls']:
                            row.append(float(result[metric]))
                        row.append(((sum(row[1:4])/3+sum(row[4:7])))/4)
                        row=[row[0]]+["{:.2f}".format(round(r*100,2)) for r in row[1:]]
                    else:
                        row=row+[""]*6
                        print(model_map[key])
                    f.write(",".join(row)+"\n")

../results/bart-large-xsum-samsum_ablation_test1_virtscribe_asr_train_ori_division_combined.csv.json
../results/bart-large-xsum-samsum_ablation_test1_virtscribe_humantrans_train_ori_division_combined.csv.json
../results/bart-large-xsum-samsum_ablation_test2_virtscribe_asr_train_ori_division_combined.csv.json
../results/bart-large-xsum-samsum_ablation_test2_virtscribe_humantrans_train_ori_division_combined.csv.json
../results/bart-large-xsum-samsum_ablation_test3_virtscribe_asr_train_ori_division_combined.csv.json
../results/bart-large-xsum-samsum_ablation_test3_virtscribe_humantrans_train_ori_division_combined.csv.json


# sample analysis

In [ ]:
from glob import glob
from tabulate import tabulate
from texttable import Texttable
from latextable import draw_latex
import json
import os

files=glob("../results/*.json")
# read a particular example
value_key="cc-Left knee pain(support:1)"

dataset="valid"
model_map={
        "longest spearker turn":'../results/longest_speaker_turn_{}.csv.json'.format(dataset),
        "longest doctor turn":'../results/longest_doctor_turn_{}.csv.json'.format(dataset),
        "12 speaker turns":'../results/12_speaker_turns_{}.csv.json'.format(dataset),
        "12 doctor turns": '../results/12_doctor_turns_{}.csv.json'.format(dataset),
        "transcript":"../results/transcript_{}.csv.json".format(dataset),
        "train_UMLS":"../results/UMLS_similarity_{}.csv.json".format(dataset),
        "train_sent":'../results/spacy_similarity_{}.csv.json'.format(dataset),
    }

for model in ["BART_large","bart-large-xsum-samsum","BioBART","LED_1024","LED_pubmed_1024"]:
        outfile='../results/{}_{}_full_ori.csv.json'.format(model,dataset)
        if outfile not in files:
            outfile=[file for file in files if "{}_{}_full_ori_prediction".format(model,dataset) in file]
            outfile.sort()
            outfile=outfile[-1]
        key=model.replace("_1024","").replace("bart-large","BART").replace("_large","").replace("-xsum-samsum","+FT_$_{\mathrm{SAMSum}}$").replace("_pubmed","+FT$_{\mathrm{PubMed}}$")
        #baseline
        model_map[key]=outfile
        
        outfile=[file for file in files if model in file and "{}_{}_ori_division_combined".format(model,dataset) in file]
        model_map[key+" (Division)"]=outfile[-1]
        
with open("../tables/example.csv","w") as f:
        headers=['Model','ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'BERTScore', 'BLEURT','Fact',"average"]
        f.write(",".join(headers)+"\n")
        table = Texttable()
        table.set_cols_align(["c"] * len(headers))
        table.set_deco(Texttable.HEADER)
        table.add_row(headers)
        for key in model_map:
            if key in ["BART","LED"]:
                f.write("{}-based\n".format(key))
            elif key=="longest spearker turn":
                f.write("Transcript-based\n")
            elif key=="train_UMLS":
                f.write("Retrieval-based\n")
            row=[key]
            if os.path.isfile(model_map[key]):
                result=json.loads(open(model_map[key]).read())[value_key]
                for metric in ['rouge1', 'rouge2', 'rougeLsum', 'bertscore-f1', 'bleurt','umls']:
                    row.append(float(result[metric]))
                row.append(((sum(row[1:4])/3+sum(row[4:7])))/4)
                row=[row[0]]+["{:.2f}".format(round(r*100,2)) for r in row[1:]]
            else:
                row=row+["NA"]*7
            table.add_row(row)
            f.write(",".join(row)+"\n")

In [ ]:
# print txt document
import pandas as pd
source_df=pd.read_csv("/home/velvinfu/data/CLEF/clef2023_v5/challenge_data/valid.csv")

text_dic={}
text_dic["true"]=source_df["note"][12]
text_dic["transcript"]=source_df["dialogue"][12]
for key in model_map:
    pred_file=model_map[key]
    source_path="/home/velvinfu/code/seq2seq/predictions/{}".format(pred_file.replace(".json","").split("/")[-1])
    df=pd.read_csv(source_path)#["note"]
    text_dic[key]=df["note"].to_list()[12]

for key in text_dic:
    with open("../example_predictions - valid_D2N080/{}.txt".format(key),"w") as f:
        f.write(text_dic[key])

In [ ]:
from glob import glob
# print all umls conccepts
#define the fact-based extractor
quickumls_fp = "/home/velvinfu/data/2022AA/des/"
umls_zip= "/home/velvinfu/data/2022AA/umls-2022AA-metathesaurus.zip"
PYM_DIR="/home/velvinfu/code_reproduction/data_augmentation/pym.sqlite3" #PyMedTermino
#the window size for transitioning
WINDOW_SIZE=5
COUNT_THRESHOLD=50
ENCODING="utf-8"

import spacy
nlp = spacy.load("en_ner_bc5cdr_md")


from semantics import SEMANTICS

from quickumls import QuickUMLS
matcher = QuickUMLS(quickumls_fp,window=WINDOW_SIZE,threshold=1,accepted_semtypes=SEMANTICS)


def add_umls(text):
    matches=matcher.match(text, ignore_syntax=True)
    UMLS_set=[]
    for match in matches:
        #print([m['semtypes'] for m in match])
        UMLS_set.append([match[0]['start'],match[0]['end'],", ".join(set([w for m in match for w in m['semtypes']]))])
        #print(match)
    UMLS_set.sort(key = lambda x: [x[0],x[1]])

    result=text[:UMLS_set[0][0]]
    for i,(s,e,type) in enumerate(UMLS_set):
        result+="<{}>".format(text[s:e])
        if i<len(UMLS_set)-1:
            result+=text[e:UMLS_set[i+1][0]]
        else:
            result+=text[e:]
    return result

for file in glob("../example_predictions - valid_D2N080/*.txt"):
    text=open(file).read()
    with open(file,"w") as f:
        f.write(add_umls(text))